# Object References, Mutability and Recycling

## Identity, Equality and Aliases

In [1]:
joe = {'name': 'Joe Biden', 'born': 1942}
president = joe
counterfeit = {'name': 'Joe Biden', 'born': 1942}

In [2]:
joe == president

True

In [3]:
joe is president

True

In [4]:
joe == counterfeit

True

In [5]:
joe is counterfeit

False

In this example `president` is an alias for `joe`, this means that both those variables points to the same object underneath. `counterfeit` however has the same value as `joe` and `president` but points to different object. This results in for example: changes applied to `joe` would also apply to `president` but not to `counterfeit`

In [6]:
joe["vice"] = "Kamala"

In [7]:
joe

{'name': 'Joe Biden', 'born': 1942, 'vice': 'Kamala'}

In [8]:
president

{'name': 'Joe Biden', 'born': 1942, 'vice': 'Kamala'}

In [9]:
counterfeit

{'name': 'Joe Biden', 'born': 1942}

## Choosing between == and is

`is` operator compares ids of objects while `==` compares values they hold.

Using `is` makes sense when comparing variable to a singleton. eg. `x is None` or `x is not None`.

When comparing singletons `is` is faster than `==`, because it cannot be overloaded, so python dont have to find and invoke any special methods to evaluate it.

## Deep and Shallow copies

Copies in python are **shallow** by default.

The easiest way to copy a list or most build-in mutable collections is to use the build-in constructor.

In [14]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)

In [15]:
print(l1 == l2)
print(l1 is l2)

True
False


This creates a **shallow** copy - the outermost container is duplicated and filled with references to the same items held by the orginal container. This saves memory and works fine when all items are **immutable**.

In [16]:
l1.append(100)
print(l1)
print(l2)

[3, [55, 44], (7, 8, 9), 100]
[3, [55, 44], (7, 8, 9)]


In [17]:
l1[1].append(999)
print(l1)
print(l2)

[3, [55, 44, 999], (7, 8, 9), 100]
[3, [55, 44, 999], (7, 8, 9)]


In [18]:
l2[1] += [33, 22]
l2[2] += (10, 11)
print(l1)
print(l2)

[3, [55, 44, 999, 33, 22], (7, 8, 9), 100]
[3, [55, 44, 999, 33, 22], (7, 8, 9, 10, 11)]


The `copy` module provides `deepcopy` and `copy` functions for arbitrary objects.

In [26]:
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)

In [27]:
import copy
bus1 = Bus(['Joe', 'Donald', 'Eliot', 'Bob'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print(f"b1: {id(bus1)}, b2: {id(bus2)}, b3: {id(bus3)}")

b1: 3002182273808, b2: 3004315909904, b3: 3004329113232


In [28]:
bus1.drop('Joe')
print(f"bus1: {bus1.passengers}")
print(f"bus2: {bus2.passengers}")
print(f"bus3: {bus3.passengers}")

bus1: ['Donald', 'Eliot', 'Bob']
bus2: ['Donald', 'Eliot', 'Bob']
bus3: ['Joe', 'Donald', 'Eliot', 'Bob']


The behavior of `copy` and `deepcopy` can be controlled by implementing `__copy__()` and `__deepcopy__()` method

## Function Parameters as References

The **only** mode of parameter passing in python is **call by sharing**, this means that each formal parameter of the function gets a copy of each reference in the arguments. In other words, the parameters inside the function become aliases of the actual args.

As the result of this, function may **change** any **mutable** object passed to it as a parameter, but it cannot change the identity of the object.

In [29]:
def f(a, b):
    a += b
    return a

In [31]:
x = 1
y = 2
print(f(x, y))
print(f"x: {x}, y: {y}")

3
x: 1, y: 2


In [32]:
a = [2, 1]
b = [3, 7]
print(f(a, b))
print(f"a: {a}, b: {b}")

[2, 1, 3, 7]
a: [2, 1, 3, 7], b: [3, 7]


## Mutable Types as Paremeter Defaults - Bad Idea

In [33]:
class HautedBus:
    def __init__(self, passengers=[]):
        self.passengers = passengers
    def pick(self, name):
        self.passengers.append(name)
    def drop(self, name):
        self.passengers.remove(name)

In [35]:
bus1 = HautedBus()
bus1.pick("Steve")

In [37]:
bus2 = HautedBus()
print(bus2.passengers)

['Steve']


This works this way beacause when `HautedBus` is instanciated empty, then `self.passengers` becomes an alias for the default value. And later when another class is instanciated with default value it receives the same reference to the default value as all previous instances. 

That is why `None` is commonly used as default value for parameters that may receive mutable values.

Unless a method is explicitly intended to mutate an object received as an argument, it should be strongly considered if aliasing the argument is the best option instead of making a copy of it.

## del and Garbage Collection

* `del` is a statement, not a function
* `del` deletes references, not objects (object may be deleted as a consequence of being unreachable after using `del`)

In CPython, the primary algorithm for garbage collection is reference counting.
